# Домашняя работа #2

## Пункт 1

Дано ОДУ:

$-u^{''}(x) + u(x) = x,\;\;0\le x \le 1,\;\;u(0)=u(1)=0$

Его аналитическое решение:

$u(x) = -\frac{e^2x - x + e^{1-x} - e^{x + 1}}{1 - e^2}$

## Пункт 2

Решим задачу численно: разобьем отрезок $[0, 1]$ на n + 1 частей одинаковой длины $h = \frac{1}{n + 1}$. В каждой точке аппроксимация 2 производной будет равна $-u^{''}(x_i)=-h^{-2}[u(x_{i-1}) - 2u(x_{i}) + u(x_{i+1})]$. Тогда можно получить следующее СЛАУ:

$Au=b$

Где $u=(u(x_1)...u(x_n))^T$, $b=(h...nh)^T$, матрица А -- трехленточная, на главной диагонали элементы равны $2h^{-2} + 1$, на диагонали выше и ниже находятся $-h^{-2}$.

Решим метод 2 способами: обычным методом гауса -- np.solve и напишем свой метод прогонки, который учитывает структуру матрицы. Также нужно будет проверить, что наш метод работает корректно, для этого сравним получившееся решение с аналитическим.

In [1]:
import numpy as np


def numpy_solve(A, b):
    return np.linalg.solve(A, b)


def thomas_solve(A, b):
    n = len(b)
    c_i = A[0][0]
    a_i, b_i = -A[1][0], -A[0][1]
    alpha = [0]
    betta = [0]

    # getting alpha and betta
    for i in range(n):
        alpha.append(b_i / (c_i - alpha[-1] * a_i))
        betta.append((b[i] + betta[-1] * a_i) / (c_i - alpha[-2] * a_i))
    alpha.pop(0)
    betta.pop(0)

    # getting answer
    x = [betta[-1]]
    for i in range(n - 2, -1, -1):
        x.append(x[-1] * alpha[i] + betta[i])
    x.reverse()
    return np.array(x)


def func(x):
    return -(np.exp(2) * x - x + np.exp(1 - x) - np.exp(x + 1)) / (1 - np.exp(2))


def get_accuracy(x, u):
    u_true = func(x)
    tol = 1e-1
    while np.allclose(u, u_true, atol=tol) and tol > 1e-17:
        tol /= 10
    return tol * 10


def generate_sys(n=1000):
    h = 1 / (n + 1)
    b = np.arange(1, n + 1) * h
    A = np.diagflat([2 * h ** (-2) + 1 for i in range(1, n + 1)]) \
        + np.diagflat([-h ** (-2) for i in range(1, n)], k=-1) \
        + np.diagflat([-h ** (-2) for i in range(1, n)], k=1)
    return A, b

A, b = generate_sys(n=1000)

sol_true = func(b)
sol_np = numpy_solve(A, b)
sol_thomas = thomas_solve(A, b)

print(f'numpy solve accuracy: {get_accuracy(b, sol_np)}')
print(f'thomas solve accuracy: {get_accuracy(b, sol_thomas)}')

numpy solve accuracy: 1.0000000000000001e-16
thomas solve accuracy: 1.0000000000000001e-16


Точность решений на 1000 точках сопоставима с машинной точностью, далее считать точность не имеет смысла -- мы и так можем сделать вывод, что решение строится корректно, поэтому цикл останавливается на этом значении.

Теперь построим графики заивисимости времени работы методов от n:

In [ ]:
from timeit import default_timer as timer
import plotly.graph_objs as go
import plotly


plotly.offline.init_notebook_mode()


def test_time(method, l=10, r=1000, step=1):
    sz = []
    elapsed_time = []
    for i in range(l, r + 1, step):
        A, b = generate_sys(i)
        start_time = timer()
        method(A, b)
        elapsed_time.append(timer() - start_time)
        sz.append(i)
    return sz, elapsed_time


def plot(x, y, names):
    traces = []
    method_colors = {
        'thomas': {'color': 'rgba(0, 255, 0, 0.8)'},
        'gauss': {'color': 'rgba(255, 0, 0, 0.8)'}
    }
    for i in range(len(names)):
        traces.append(
            go.Scatter(
                x=x[i],
                y=y[i],
                mode='lines+markers',
                marker=method_colors[names[i]],
                name=names[i]
            )
        )

    layout = go.Layout(
        title='Зависимость времени работы от n',
        xaxis={
            'title': 'n'
        },
        yaxis={
            'title': 'time, ms'
        }
    )

    fig = go.Figure(data=traces, layout=layout)
    plotly.offline.iplot(fig)
    

r = 3000
step = 30
x_g, y_g = test_time(numpy_solve, l=10, r=r, step=step)
x_t, y_t = test_time(thomas_solve, l=10, r=r, step=step)

plot([x_g], [y_g], names=['gauss'])
plot([x_t], [y_t], names=['thomas'])
plot([x_g, x_t], [y_g, y_t], names=['gauss', 'thomas'])

Как и ожидалось, метод прогонки работает значительно быстрее, ибо в нем учитывается структура, что позволяет решить систему за линейное время, по сравнению с кубическим, за которое работает Гаусс. Форма графиков также подтверждает только что сказанное -- график гаусса начинает возрастать кубически при увеличении n, а график метода прогонки растет линейно.